In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go

# Sim some financial data
np.random.seed(7)
dates = pd.date_range(start='2024-01-01', periods=100)
data = np.random.randn(100).cumsum()
df = pd.DataFrame(data, columns=['Price'], index=dates)

# Prepare dataset for training
window_size = 5
features = []
labels = []

for i in range(window_size, len(df)):
    features.append(df.iloc[i-window_size:i, 0])
    labels.append(df.iloc[i, 0])

features, labels = np.array(features), np.array(labels)

# Split the dataset into training & testing
split = int(len(features) * 0.8)
train_features, test_features = features[:split], features[split:]
train_labels, test_labels = labels[:split], labels[split:]

# Build a simple LSTM model
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(window_size,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_features, train_labels, epochs=10, batch_size=1)

# Make predictions
predictions = model.predict(test_features)

# Visualize the results
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates[split+window_size:], y=test_labels, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=dates[split+window_size:], y=predictions.flatten(), mode='lines', name='Predicted'))

fig.update_layout(title='Financial Data Prediction',
                                  xaxis_title='Date',
                                  yaxis_title='Price')

fig.show()

Epoch 1/10


c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 997us/step - loss: 15.6095
Epoch 2/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - loss: 1.2619
Epoch 3/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step - loss: 1.4199
Epoch 4/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - loss: 1.9803
Epoch 5/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 1.2762
Epoch 6/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step - loss: 1.0038
Epoch 7/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 631us/step - loss: 1.0587
Epoch 8/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 1.2315
Epoch 9/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - loss: 1.0045
Epoch 10/10
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - loss: 0.8387
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [2]:
# Instalar bibliotecas necessárias (no terminal):
# !pip install stable-baselines3 gym pandas numpy

import gym
import numpy as np
import pandas as pd
from stable_baselines3 import PPO
from gym import spaces

# Criar o ambiente personalizado
class TradingEnv(gym.Env):
    def __init__(self, df):
        super(TradingEnv, self).__init__()
        self.df = df
        self.current_step = 0
        self.balance = 1000  # Saldo inicial
        self.position = 0  # Posição atual (quantidade de ações compradas)
        
        # Definir espaços de ação e observação
        self.action_space = spaces.Discrete(3)  # 0 = Manter, 1 = Comprar, 2 = Vender
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.balance = 1000
        self.position = 0
        return [self.df.iloc[self.current_step]]

    def step(self, action):
        # Aplicar a ação escolhida
        current_price = self.df.iloc[self.current_step]
        reward = 0
        
        if action == 1:  # Comprar
            self.position += 1
            self.balance -= current_price
        elif action == 2:  # Vender
            if self.position > 0:
                self.position -= 1
                self.balance += current_price
                reward = 1  # Recompensa por obter lucro ao vender
        
        # Atualizar o passo atual
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1

        # Calcular o valor da carteira
        total_value = self.balance + self.position * current_price
        
        # Recompensa baseada no valor da carteira
        if not done:
            reward += (total_value - 1000) / 1000  # Recompensa relativa ao saldo inicial

        # Obter próximo estado
        next_state = [self.df.iloc[self.current_step]]
        return next_state, reward, done, {}

# Simular dados financeiros
np.random.seed(7)
dates = pd.date_range(start='2024-01-01', periods=100)
data = np.random.randn(100).cumsum() + 100
df = pd.Series(data, index=dates)

# Criar o ambiente
env = TradingEnv(df)

# Treinar o agente com PPO
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)

# Testar o agente
done = False
obs = env.reset()
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    print(f"Ação: {action}, Recompensa: {rewards}")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\fabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning:

You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.



---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | 26.9     |
| time/              |          |
|    fps             | 484      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
Ação: 1, Recompensa: 0.0
Ação: 2, Recompensa: 0.9995340626294591
Ação: 0, Recompensa: -0.000465937370540928
Ação: 0, Recompensa: -0.000465937370540928
Ação: 2, Recompensa: -0.000465937370540928
Ação: 2, Recompensa: -0.000465937370540928
Ação: 2, Recompensa: -0.000465937370540928
Ação: 1, Recompensa: -0.000465937370540928
Ação: 2, Recompensa: 1.0005517206351227
Ação: 2, Recompensa: 0.0005517206351225923
Ação: 0, Recompensa: 0.0005517206351225923
Ação: 0, Recompensa: 0.0005517206351225923
Ação: 2, Recompensa: 0.0005517206351225923
Ação: 2, Recompensa: 0.0005517206351225923
Ação: 2, Recompensa: 0.0005517206351225923
Ação: 2, Recompensa: 0.0005517206351225

In [3]:
# Instalar bibliotecas necessárias (no terminal):
# !pip install gym pandas numpy torch

import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from gym import spaces

# Criar o ambiente personalizado
class TradingEnv(gym.Env):
    def __init__(self, df):
        super(TradingEnv, self).__init__()
        self.df = df
        self.current_step = 0
        self.balance = 1000  # Saldo inicial
        self.position = 0  # Posição atual (quantidade de ações compradas)
        
        # Definir espaços de ação e observação
        self.action_space = spaces.Discrete(3)  # 0 = Manter, 1 = Comprar, 2 = Vender
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(len(df.columns),), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.balance = 1000
        self.position = 0
        return self.df.iloc[self.current_step].values  # Retornar todos os indicadores como observação

    def step(self, action):
        # Aplicar a ação escolhida
        current_price = self.df['Price'].iloc[self.current_step]
        reward = 0
        
        if action == 1:  # Comprar
            self.position += 1
            self.balance -= current_price
        elif action == 2:  # Vender
            if self.position > 0:
                self.position -= 1
                self.balance += current_price
                reward = current_price / 100  # Recompensa proporcional ao preço de venda
        
        # Atualizar o passo atual
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1

        # Calcular o valor da carteira
        total_value = self.balance + self.position * current_price
        
        # Recompensa baseada no valor da carteira
        if not done:
            reward += (total_value - 1000) / 1000  # Recompensa relativa ao saldo inicial

        # Obter próximo estado
        next_state = self.df.iloc[self.current_step].values
        return next_state, reward, done, {}

# Simular dados financeiros com múltiplos indicadores
np.random.seed(7)
dates = pd.date_range(start='2024-01-01', periods=100)
data = np.random.randn(100).cumsum() + 100
volume = np.random.randint(100, 200, size=100)
ma = pd.Series(data).rolling(window=5).mean().fillna(0)
df = pd.DataFrame({'Price': data, 'Volume': volume, 'Moving_Avg': ma}, index=dates)

# Criar o ambiente
env = TradingEnv(df)

# Criar o modelo de rede neural em PyTorch
class TradingAgent(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TradingAgent, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.output = nn.Linear(64, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.output(x)

# Definir parâmetros do modelo
input_dim = len(df.columns)
output_dim = 3  # 3 ações possíveis
model = TradingAgent(input_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Treinar o agente manualmente
num_episodes = 10
epsilon = 1.0  # Fator de exploração
epsilon_decay = 0.99
for episode in range(num_episodes):
    obs = torch.tensor(env.reset(), dtype=torch.float32).unsqueeze(0)  # Ajustar a dimensão para (1, input_dim)
    done = False
    steps = 0
    while not done and steps < 200:
        # Escolher uma ação usando epsilon-greedy
        if np.random.rand() < epsilon:
            action = env.action_space.sample()  # Exploração
        else:
            q_values = model(obs)
            action = torch.argmax(q_values).item()  # Exploração dirigida
        
        # Executar a ação no ambiente
        next_obs, reward, done, _ = env.step(action)
        next_obs = torch.tensor(next_obs, dtype=torch.float32).unsqueeze(0)  # Ajustar a dimensão para (1, input_dim)
        
        # Atualizar o modelo usando o erro TD
        target = reward + 0.95 * torch.max(model(next_obs)).item() if not done else reward
        target_f = model(obs).detach().clone()
        target_f[0, action] = target
        
        # Calcular a perda e fazer o backpropagation
        output = model(obs)
        loss = loss_fn(output, target_f)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        obs = next_obs
        steps += 1
    
    # Decaimento do epsilon
    epsilon *= epsilon_decay

# Testar o agente
done = False
obs = torch.tensor(env.reset(), dtype=torch.float32).unsqueeze(0)  # Ajustar a dimensão para (1, input_dim)
steps = 0
while not done and steps < 200:
    q_values = model(obs)
    action = torch.argmax(q_values).item()
    obs, reward, done, _ = env.step(action)
    obs = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)  # Ajustar a dimensão para (1, input_dim)
    print(f"Ação: {action}, Recompensa: {reward}")

Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
Ação: 0, Recompensa: 0.0
